1. Write SQL statements to create below objects 
 - A catalog named telecom_catalog_assign </br>
 - A schema landing_zone <br>
 - A volume landing_vol <br>


In [0]:
%sql
create catalog if not exists telecom_catalog_assign;


In [0]:
%sql

create schema if not exists telecom_catalog_assign.landing_zone;


In [0]:
%sql
create volume  if not exists telecom_catalog_assign.landing_zone.landing_vol;

2. Using dbutils.fs.mkdirs, create below folders<br>
- /Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/ <br>
- /Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/ <br>
- /Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower<br>



In [0]:
dbutils.fs.mkdirs("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
dbutils.fs.mkdirs("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
dbutils.fs.mkdirs("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")
dbutils.fs.mkdirs("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")

#### Create the customer , usage and tower logs

In [0]:

customer_csv = ''' 101,Arun,31,Chennai,PREPAID 102,Meera,45,Bangalore,POSTPAID 103,Irfan,29,Hyderabad,PREPAID 104,Raj,52,Mumbai,POSTPAID 105,,27,Delhi,PREPAID 106,Sneha,abc,Pune,PREPAID '''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count 101\t320\t1500\t20 102\t120\t4000\t5 103\t540\t600\t52 104\t45\t200\t2 105\t0\t0\t0 '''

tower_logs_region = '''event_id|customer_id|tower_id|signal_strength|timestamp 5001|101|TWR01|-80|2025-01-10 10:21:54 5004|104|TWR05|-75|2025-01-10 11:01:12 '''



In [0]:

dbutils.fs.put("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", customer_csv, True)

In [0]:
# usage_tsv 
dbutils.fs.put("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv",usage_tsv,True)

In [0]:

dbutils.fs.put("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1",tower_logs_region,True)
dbutils.fs.put("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2",tower_logs_region,True)


##Driver program

In [0]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.appName("usecase3").getOrCreate();


In [0]:
dbutils.fs.head("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")

In [0]:
schema = "customer_id string,name string,age integer,city string,plan string"
df = sparkSession.read.csv(path="dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv",sep=',',schema=schema)
display(df)

In [0]:
from pyspark.sql.functions import col
def getlist():
    customer_csv = ''' 101,Arun,31,Chennai,PREPAID 102,Meera,45,Bangalore,POSTPAID 103,Irfan,29,Hyderabad,PREPAID 104,Raj,52,Mumbai,POSTPAID 105,,27,Delhi,PREPAID 106,Sneha,abc,Pune,PREPAID ''';
    data = []
    customer_csv_split = customer_csv.strip().split(" ")
    for item in customer_csv_split:
        data.append(tuple(item.split(",")))
    return data;

    data = getlist();

df = sparkSession.createDataFrame(data,schema=["customer_id","name","age","city","plan"]);
display(df)


In [0]:
# dbutils.notebook.run('/Workspace/Users/anto2003.sfn@gmail.com/Usecase_3_solution',60)

In [0]:
# 3.1 Read all tower logs using: Path glob filter (example: *.csv) Multiple paths input Recursive lookup
spark.read.csv(path=["dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/"],header=True,inferSchema=True,sep="|",pathGlobFilter="*.csv",recursiveFileLookup=True).show()

# #3.2 Demonstrate these 3 reads separately: Using pathGlobFilter Using list of paths in spark.read.csv([path1, path2]) Using .option("recursiveFileLookup","true")
# spark.read.option("header","True").option("inferSchema","True").option("sep","|").option("pathGlobFilter","*.csv").option("recursiveFileLookup","True").format("CSV").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/").show()
